<a href="https://colab.research.google.com/github/Expert-Han/Nonlinear_Model_Predictive_Control_Python_byHan/blob/main/NMPC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Python version check in Colab & Casadi install

In [2]:
!python --version

Python 3.8.10


In [3]:
pip install casadi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.2/34.2 MB 53.0 MB/s eta 0:00:00


In [23]:
from casadi import *
x = MX.sym("x")
print(jacobian(sin(x),x))


cos(x)
range(0, 3)


## Main code for NMPC (Nonlinear Model Predictive Control)

In [20]:
import numpy as np

## Predict u code 

def  solve_CFTOCP(x_t, N, Q, R, ti):
  M = 0.5       # mass of the cart (kg)  
  m = 0.5       # mass of the pendulum bob (kg)
  L = 1         # length of the pendulum rod (m)
  g = 9.81      # gravity
  im = 0.5      # inertia  
  size_n = 4

  x1 = MX.sym('x1') # States define
  x2 = MX.sym('x2')
  x3 = MX.sym('x3')
  x4 = MX.sym('x4')

  x  = [[x1], [x2], [x3], [x4]] 
  u  = MX.sym('u'); # Controls

  # Inverted Pendulum
  dx1 = x2
  dx2 = (-m**2*(L/2)**2*np.sin(x1)*np.cos(x1)*x2**2+(M+m)*m*g*(L/2)*np.sin(x1)+m*(L/2)*np.cos(x1)*u)/((M+m)*(im+m*(L/2)**2)-m**2*(L/2)**2*np.cos(x1)**2)
  dx3 = x4
  dx4 = (m**2*(L/2)**2*np.sin(x1)*np.cos(x1)*g-m*(im+m*(L/2)**2)*(L/2)*np.sin(x1)*x2**2+(im+m*(L/2)**2)*u)/((M+m)*(im+m*(L/2)**2)-m**2*(L/2)**2*np.cos(x1)**2)

  f = Function('f',{x,u},{dx1,dx2,dx3,dx4},{'ode'})
  intg_options = struct
  intg_options.tf = ti
  intg_options.simplify = true

  dae = struct
  dae.x = x        # What are states?
  dae.p = u         # What are parameters (=fixed during the integration horizon)?
  dae.ode = f(x,u)  # Expression for the right-hand side

  intg = integrator('intzg','rk',dae,intg_options)
  res = intg('x0',x,'p',u) # Evaluate with symbols
  x_next = res.xf
  F = Function('F',{x,u},{x_next},{'x','u'},{'x_next'})    

  opti = casadi.Opti()

  # Prediction Size N
  tn = N/ti
  x = opti.variable(4,tn+1) # Decision variables for state trajetcory, tn + next_state
  u = opti.variable(1,tn)
  p = opti.parameter(4,1)  # Parameter (not optimized over) <=> updated initial state
  
  opti.subject_to(x[:,1]==p)
  # Constraints
  # Select state and input constraints matrices from polyhedrons
  #Hx  = X.A;  bx  = X.b;
  #Hu  = U.A;  bu  = U.b;

  for k in range(0, tn+1):
    opti.subject_to(x[:,k+1]==F(x[:,k],u[:,k]))
 
  for k in range(0, tn+2):
    opti.subject_to(x[0,k] <= 1) # The first state is bounded by 1

  for k in range(0, tn+1):
      opti.subject_to(u[1,k] <= 10)  # The first input is bounded by 10

  # Cost
  Cost=0
 
  for k in range(0, tn+1):
      Cost = Cost + x[:,k].T*Q*x[:,k] + u[:,k].T*R*u[:,k] 
  
  Cost = Cost + x[:,tn+1].T*Q*x[:,tn+1] # Add final state

  # Solve
  opti.minimize(Cost)
  opti.set_value(p,x_t)
  opti.solver('ipopt')
  sol = opti.solve()

  x = sol.value(x)
  u = sol.value(u)

  x = double(x)
  u = double(u)

  uPred = u[:,1]
 
  return uPred

## Simulation Code

# Q = np.array([1, 0, 0, 0],[0, 1, 0, 0],[0, 0, 1, 0],[0, 0, 0, 1]) # weighting matrix for state
# R = np.array([0.1]) # weighting matrix for input
Q = [[1, 0, 0, 0],[0, 1, 0, 0],[0, 0, 1, 0],[0, 0, 0, 1]] # weighting matrix for state
R = [0.1] # weighting matrix for input
N = 5 # Prediction Time

# x_cl_out{1} = x_cl{1};
# u_cl_out{1} = u_cl{1};

M = 0.5       # mass of the cart (kg)  
m = 0.5       # mass of the pendulum bob (kg)
L = 1         # length of the pendulum rod (m)
g = 9.81      # gravity
im = 0.5      # inertia  

x1 = MX.sym('x1') # States define
x2 = MX.sym('x2')
x3 = MX.sym('x3')
x4 = MX.sym('x4')

x  = [[x1], [x2], [x3], [x4]]  
# x  = np.array([x1, x2, x3, x4]);    
u  = MX.sym('u') # Controls

ti = 0.001 # RK45 sampling time

# Inverted Pendulum
dx1 = x2
dx2 = (-m**2*(L/2)**2*np.sin(x1)*np.cos(x1)*x2**2+(M+m)*m*g*(L/2)*np.sin(x1)+m*(L/2)*np.cos(x1)*u)/((M+m)*(im+m*(L/2)**2)-m**2*(L/2)**2*np.cos(x1)**2)
dx3 = x4
dx4 = (m**2*(L/2)**2*np.sin(x1)*np.cos(x1)*g-m*(im+m*(L/2)**2)*(L/2)*np.sin(x1)*x2**2+(im+m*(L/2)**2)*u)/((M+m)*(im+m*(L/2)**2)-m**2*(L/2)**2*np.cos(x1)**2)

f = Function('f',{x,u},{dx1,dx2,dx3,dx4},{'ode'})
intg_options = struct
intg_options.tf = ti
intg_options.simplify = true

dae = struct
dae.x = x         # What are states?
dae.p = u         # What are parameters (=fixed during the integration horizon)?
dae.ode = f(x,u)  # Expression for the right-hand side

intg = integrator('intzg','rk',dae,intg_options)
res = intg('x0',x,'p',u) # Evaluate with symbols
x_next = res.xf
F = Function('F',{x,u},{x_next},{'x','u'},{'x_next'})  

# Initialize simulation parameters
i = 0    
degree = 180
x0=np.array([degree*(np.pi)/180, 0, 0, 0], dtype=np.float64) 
x_sig = np.array([x0], dtype=np.float64)
uPred = solve_CFTOCP(x_sig[i], N, Q, R, ti)
u_sig = np.array([uPred], dtype=np.float64)

tollerance = 10^(-9) # Terminating condition for NMPC <=> x'x <= tollerance
exitFlag = 0

while exitFlag == 0:
  # uPred = solve_CFTOCP(x_NMPC(:,t), N, Q, R, X, U, ti, hmax); # X, U is constraint set 
  uPred = solve_CFTOCP(x_sig[i], N, Q, R, ti)
  dx = np.full(F(x_sig[i],uPred))
  x_sig = np.vstack((x_sig,dx))
  if i >= 1:
    u_sig = np.vstack((u_sig,uPred))    
  i = i + 1
  # Check terminating condition
  if x_sig[i].T*x_sig[i] < tollerance:
    exitFlag = 1

plt.figure()
plt.plot(tspan, x_sig[:, 0], label="x1")
#plt.plot(tspan, x_sig[:, 1], label="x2")
#plt.plot(tspan, x_sig[:, 2], label="x3")
#plt.plot(tspan, x_sig[:, 3], label="x3")
plt.grid()
plt.xlabel("Time")
plt.ylabel("State response")
plt.legend()
plt.show()

plt.figure()
plt.plot(tspan, u_sig[:, 0], label="u")
plt.grid()
plt.xlabel("Time")
plt.ylabel("Control signal")
plt.legend()
plt.show()